In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pyBigWig


In [4]:
'''
Created on May 3, 2016

@author: Gabe
'''
import pyBigWig
import numpy as np

class ReadDensity():
    """
    ReadDensity class
    Attributes:
        self.pos(positive *.bw file)
        self.neg(negative *.bw file)
    """
    def __init__(self, pos, neg, name = None):
        try:
            self.pos = pyBigWig.open(pos)
            self.neg = pyBigWig.open(neg)
            self.name = name if name is not None else pos.replace('pos','*').replace('neg','*')
        except Exception as e:
            print("couldn't open the bigwig files!")
            print(e)
            return 1
    def get_name(self):
        """
        returns name
        """
        return self.name
    
    def values(self, chrom, start, end, strand):
        """
        Given a chromosome coordinate, return a list of values
        pertaining to the rbpmaps over each nucleotide position.
        Reverse the list if going in the negative strand.
        
        Args:
            chrom (str): (eg. chr1)
            start (int): 0-based start (first position in chromosome is 0)
            end (int): 1-based end (last position is not included)
            strand (char): either '+' or '-'
        """
        try:
            if strand == "+":
                return self.pos.values(chrom, start, end)
            elif strand == "-":
                return list(reversed(self.neg.values(chrom, start, end)))
            else:
                raise("Strand neither + or -")
        except RuntimeError:
            # usually occurs when no chromosome exists in the bigwig file
            return [np.NaN]*abs(start-end)


In [5]:
'''
Created on Sep 21, 2016

@author: brian
'''
import pybedtools as bt

class Feature():
    '''
    classdocs
    '''


    def __init__(self, annotation, source):
        '''
        Constructor
        '''
        self.annotation = annotation.rstrip()
        self.source = source

    def get_bedtool(self):
        if(self.source == 'bed'):
            chrom, start, end, name, score, strand = self.annotation.split('\t')
        return bt.create_interval_from_list([chrom,
                                             start,
                                             end,
                                             name,
                                             score,
                                             strand])

"""        
annotation = 'chr3:53274267:53274364:-@chr3:53271813:53271836:-@chr3:53268999:53269190:-'
print(annotation)
F = SkippedExonFeature(annotation, 'miso')
up, se, down = F.get_bedtools()
print(up)
print(se)
print(down)
"""

"        \nannotation = 'chr3:53274267:53274364:-@chr3:53271813:53271836:-@chr3:53268999:53269190:-'\nprint(annotation)\nF = SkippedExonFeature(annotation, 'miso')\nup, se, down = F.get_bedtools()\nprint(up)\nprint(se)\nprint(down)\n"

In [6]:
import itertools

def some_range(rbp, interval, left_flank = 0, right_flank = 0):
    if interval.strand == "+":
        wiggle = rbp.values(interval.chrom, interval.start - left_flank, interval.end + right_flank, interval.strand)
    elif interval.strand == "-":
        wiggle = rbp.values(interval.chrom, interval.start - left_flank, interval.end + right_flank, interval.strand)
    else:
        print "Strand not correct", interval.strand
        raise()
    return wiggle   
def get_scale(wiggle):
    # type: (Series) -> Series
    '''
    Returns a wiggle of any N that is divisible by 100.
    
    '''
    if(len(wiggle)==100): # no need to do any calculating.
        return wiggle
    elif len(wiggle) == 1:
        return pd.Series(list(itertools.chain.from_iterable([multiply(w) for w in wiggle])))
    elif len(wiggle) < 100: 
        wiggle = pd.Series(list(itertools.chain.from_iterable([multiply(w) for w in wiggle])))
        
    dist = [0]*100
    x = 0
    step = 0.01
    y = 0
        
    for pos, value in enumerate(wiggle):
        if(float(pos+1)/len(wiggle)) < step:
            y = y + 1
            dist[x] = dist[x] + value            
        else:
            dist[x] = dist[x] / y
                
            step = step + 0.01
            x = x + 1
            dist[x] = value
            y = 1
    dist[x] = dist[x] / y
    return(pd.Series(dist))
def rename_index(interval_name):
    # type: (str) -> str
    '''
    Reformats a BedTool Interval name into a non-tabbed format.
    '''
    chrom, start, end, name, score, strand = str(interval_name).strip().split('\t')
    return "{}:{}-{}:{}:{}".format(chrom, start, end, name, strand)


In [7]:
def normalize_and_per_region_subtract(density, input_density, min_density_threshold = 0):
    """
    Normalizes ip matrix of m x n (where m is the row of each event in a feature,
    and n is the column relating to nucleotide position). 
    """
    # PDF_CONST = 1.0/len(density.columns)
    
    df_indices = density.index
    dfi_indices = input_density.index
    missing = set(df_indices) - set(dfi_indices)
    
    input_density = input_density.append(input_density.ix[missing])
    
    pdf = calculate_pdf(density, min_density_threshold)
    pdfi = calculate_pdf(input_density, min_density_threshold)
    
    
    
    # pdfi = fill_all_nans_with_minpdf(pdfi, PDF_CONST)
    # pdf = fill_all_nans_with_minpdf(pdf, PDF_CONST)
    
    subtracted = pdf.sub(pdfi)
    # print("TYPE AFTER PER REGION SUBTRACT: {}".format(type(subtracted)))
    return subtracted

def calculate_pdf(density, min_density_threshold = 0):
    """
    Calculates the PDF of a density matrix.
    Logic:
    
    Args: 
        density (pandas.DataFrame) : r x c matrix of densities. May contain
            NaN corresponding to values in which no density was returned.
            These values should be counted.
            May contain -1 corresponding to values in which a particular
            region is shorter than the full DataFrame length. These 
            values should not be counted.
        min_density_threshold (integer) : minimum total density across
            a row. (Deprecated - may be removed in the future)
    
    Returns:
        pdf (pandas.DataFrame) : r x c matrix of densities normalized
            across each respective (r)ow as a probability density func.
    """
    density = density.fillna(0) # NaNs are regions which contain zero density
    df = density.replace(-1, np.nan) # -1 are regions which should not be counted at all
    
    # df = df[df.sum(axis=1) > min_density_threshold]
    
    min_normalized_read_number = min([item for item in df.unstack().values if item > 0])
    print(min_normalized_read_number)
    df = df + min_normalized_read_number
    pdf = df.div(df.sum(axis=1), axis=0)
    return pdf # , mean, sem

In [9]:
df = pd.read_table('/projects/ps-yeolab3/bay001/maps/archived/10-27-2016/se/204_01_RBFOX2.ip.all.feature.se.raw_density_matrix.csv',
                  sep=',',index_col=0)
dfi = pd.read_table('/projects/ps-yeolab3/bay001/maps/archived/10-27-2016/se/204_01_RBFOX2.input.all.feature.se.raw_density_matrix.csv',
                   sep=',',index_col=0)

In [10]:
X = normalize_and_per_region_subtract(df,dfi,0)

0.233893454075
0.265550494194


In [13]:
interval = bt.create_interval_from_list(['chr10','100004277','100004430','shouldbeblank','0','+'])
pos = '/projects/ps-yeolab3/encode/analysis/encode_v12/204_01_RBFOX2.merged.r2.norm.pos.bw'
neg = '/projects/ps-yeolab3/encode/analysis/encode_v12/204_01_RBFOX2.merged.r2.norm.neg.bw'
rbp = ReadDensity(neg,pos)
i = some_range(rbp,interval)

In [15]:
import pysam

In [21]:
input_bam = '/projects/ps-yeolab3/encode/analysis/encode_v12/204_01_RBFOX2.merged.r2.bam'
bam = pysam.AlignmentFile(input_bam)

In [22]:
bam.count()

4275451

In [25]:
bam.fetch(until_eof=True)

In [ ]:
%%timeit
calculate_pdf(df,0)

In [38]:
Y = normalize_and_per_region_subtract2(df, dfi, 0)

0.233893454075
0.265550494194


In [36]:
X.to_csv('~/testX.txt')
Y.to_csv('~/testY.txt')

In [11]:
! diff ~/testX.txt ~/testY.txt

In [18]:
def entropy_of_reads(density, input_density, min_density_threshold = 0):
    """
    Return the entropy of each position
    Logic: 
        Fill NaNs with zero - we want to count all regions and add pseudocount
        Fill -1 with NaNs - we want to negate any -1, which signifies a premature exon boundary
        Add minimum pseudocount
        Calculate entropy
    """
    df_indices = density.index
    dfi_indices = input_density.index
    missing = (set(df_indices) - set(dfi_indices))
    
    input_density = input_density.append(input_density.ix[missing])
    
    density = density.fillna(0) # NaNs are regions which contain zero density
    df = density.replace(-1, np.nan) # -1 are regions which should not be counted at all
    
    input_density = input_density.fillna(0) # NaNs are regions which contain zero density
    dfi = input_density.replace(-1, np.nan) # -1 are regions which should not be counted at all
    
    min_ip_read_number = min([item for item in df.unstack().values if item > 0])
    min_in_read_number = min([item for item in dfi.unstack().values if item > 0])
    min_read_number = min(min_ip_read_number,min_in_read_number)
    
    df = df + min_read_number
    dfi = dfi + min_read_number
    
    en = df.multiply(np.log2(df.div(dfi)))
    # print("TYPE AFTER ENTROPY OF READS: {}".format(type(en)))
    return en

In [30]:
x = {'a':1}

In [31]:
len(x)

1